In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
beta = 0.15
beta_f = 0.25
mu = 0.1
gamma = 0.125
sigma = 0.42
lamb = 0.17
lamb_f = 0.27
tau = 0.01
omega = 0.05
kappa = 0.21
delta = 0.13
epsilon = 0.22
phi = 0.13
phi_f = 0.23

In [ ]:
def n_update(n, mu):
    seeds = np.random.uniform(0, 1, np.int_(n))
    pop_update = np.zeros(4)
    for seed in seeds:
        if seed < mu:
            pop_update[0] += 1
    return pop_update

def s_update(s, mu, beta, beta_f):
    seeds = np.random.uniform(0, 1, np.int_(s))
    pop_update = np.zeros(4)
    for seed in seeds:
        if seed < mu:
            pop_update[0] -= 1
        elif seed < (mu + beta + beta_f):
            pop_update[0] -= 1
            pop_update[1] += 1
    return pop_update

def i_update(i, mu, gamma):
    seeds = np.random.uniform(0, 1, np.int_(i))
    pop_update = np.zeros(4)
    for seed in seeds:
        if seed < mu:
            pop_update[1] -= 1
        elif seed < (mu + gamma):
            pop_update[1] -= 1
            pop_update[3] += 1
    return pop_update

def r_update(r, mu):
    seeds = np.random.uniform(0, 1, np.int_(r))
    pop_update = np.zeros(4)
    for seed in seeds:
        if seed < mu:
            pop_update[2] -= 1
    return pop_update

def f_update(f, mu, sigma):
    seeds = np.random.uniform(0, 1, np.int_(f))
    pop_update = np.zeros(4)
    for seed in seeds:
        if seed < mu:
            pop_update[3] -= 1
        elif seed < (mu + sigma):
            pop_update[3] -= 1
            pop_update[2] += 1
    return pop_update

In [ ]:
def base_model(init_pop, time, beta, beta_f, mu, gamma, sigma):
    pop = np.zeros([4, time+1])
    pop[0,0] = init_pop
    for t in range(1, time+1):
        n = np.sum(pop, axis=0)[t-1]
        s = pop[0, t-1]
        i = pop[1, t-1]
        r = pop[2, t-1]
        f = pop[3, t-1]

        n_change = n_update(n, mu)
        s_change = s_update(s, mu, beta, beta_f)
        i_change = i_update(i, mu, gamma)
        r_change = r_update(r, mu)
        f_change = f_update(f, mu, sigma)
        
        pop_change = n_change + s_change + i_change + r_change + f_change
        pop[:,t] = np.add(pop[:,t-1], pop_change)
        
    df = pd.DataFrame({
        'time': np.arange(time+1),
        's': pop[0,:],
        'i': pop[1,:],
        'r': pop[2,:],
        'f': pop[3,:],
    })
    return df

In [ ]:
base = base_model(1000, 100, beta, beta_f, mu, gamma, sigma)
print(base)

In [ ]:
plt.figure(figsize=(9, 6))
sns.lineplot(data=base, x="time", y="s", label="Susceptible")
sns.lineplot(data=base, x="time", y="i", label="Infectious")
sns.lineplot(data=base, x="time", y="r", label="Removed")
sns.lineplot(data=base, x="time", y="f", label="Funeral")
plt.title('Stochastic Simulation of SIRF Base Model')
plt.xlabel('Time')
plt.ylabel('Population')
plt.legend(loc = "upper right");

In [ ]:
def n_update(n, mu):
    seeds = np.random.uniform(0, 1, np.int_(n))
    pop_update = np.zeros(6)
    for seed in seeds:
        if seed < mu:
            pop_update[0] += 1
    return pop_update

def s_update(s, mu, lamb, lamb_f):
    seeds = np.random.uniform(0, 1, np.int_(s))
    pop_update = np.zeros(6)
    for seed in seeds:
        if seed < mu:
            pop_update[0] -= 1
        elif seed < (mu + lamb + lamb_f):
            pop_update[0] -= 1
            pop_update[2] += 1
    return pop_update


def e_update(e, mu, beta, beta_f):
    pop_update = np.zeros(6)
    try:
        seeds = np.random.uniform(0, 1, np.int_(e))
    except ValueError:
        return pop_update
    for seed in seeds:
        if seed < mu:
            pop_update[2] -= 1
        elif seed < (mu + lamb + lamb_f):
            pop_update[2] -= 1
            pop_update[4] += 1
    return pop_update

def r_update(r, mu):
    seeds = np.random.uniform(0, 1, np.int_(r))
    pop_update = np.zeros(6)
    for seed in seeds:
        if seed < mu:
            pop_update[3] -= 1
    return pop_update

def f_update(f, mu):
    seeds = np.random.uniform(0, 1, np.int_(f))
    pop_update = np.zeros(6)
    for seed in seeds:
        if seed < mu:
            pop_update[5] -= 1
    return pop_update

def i_update(i, mu, sigma, alpha, gamma, kappa):
    seeds = np.random.uniform(0, 1, np.int_(i))
    pop_update = np.zeros(6)
    for seed in seeds:
        if seed < mu:
            pop_update[4] -= 1
        elif seed < (mu + sigma + alpha):
            pop_update[4] -= 1
            pop_update[3] += 1
        elif seed < (mu + sigma + alpha + gamma):
            pop_update[4] -= 1
            pop_update[5] += 1
        elif seed < (mu + sigma + alpha + gamma + kappa):
            pop_update[4] -= 1
            pop_update[1] += 1
    return pop_update


def h_update(h, mu, delta, tau, omega):
    seeds = np.random.uniform(0, 1, np.int_(h))
    pop_update = np.zeros(6)
    for seed in seeds:
        if seed < mu:
            pop_update[2] -= 1
        elif seed < (mu + delta):
            pop_update[2] -= 1
            pop_update[5] += 1
        elif seed < (mu + delta + tau + omega):
            pop_update[2] -= 1
            pop_update[3] +=1
    return pop_update

In [ ]:
def sherif_model(init_pop, time, beta, beta_f, lamb, lamb_f, mu, gamma, sigma, alpha, kappa, delta, tau, omega):
    
    pop = np.zeros([6, time+1])
    pop[0,0] = init_pop
    for t in range(1, time+1):
        n = np.sum(pop, axis=0)[t-1]
        s = pop[0, t-1]
        h = pop[1, t-1]
        e = pop[2, t-1]
        r = pop[3, t-1]
        i = pop[4, t-1]
        f = pop[5, t-1]

        n_change = n_update(n, mu)
        s_change = s_update(s, mu, lamb, lamb_f)
        h_change = h_update(h, mu, delta, tau, omega)
        e_change = e_update(e, mu, beta, beta_f)
        r_change = r_update(r, mu)
        i_change = i_update(i, mu, sigma, alpha, gamma, kappa)
        f_change = f_update(f, mu)
        
        pop_change = n_change + s_change + h_change + e_change + i_change + r_change + f_change
        pop[:,t] = np.add(pop[:,t-1], pop_change)
        
    df = pd.DataFrame({
        'time': np.arange(time+1),
        's': pop[0,:],
        'h': pop[1,:],
        'e': pop[2,:],
        'r': pop[3,:],
        'i': pop[4,:],
        'f': pop[5,:],

    })
    return df

In [ ]:
sherif = sherif_model(1000, 100, beta, beta_f, lamb, lamb_f, mu, gamma, 
                      sigma, alpha, kappa, delta, tau, omega)
print(sherif)

In [ ]:
plt.figure(figsize=(9, 6))
sns.lineplot(data=sherif, x="time", y="s", label="Susceptible")
sns.lineplot(data=sherif, x="time", y="h", label="Hospitalized")
sns.lineplot(data=sherif, x="time", y="e", label="Exposed")
sns.lineplot(data=sherif, x="time", y="r", label="Removed")
sns.lineplot(data=sherif, x="time", y="i", label="Infectious")
sns.lineplot(data=sherif, x="time", y="f", label="Funeral")
plt.title('Stochastic Simulation of SHERIF Model')
plt.xlabel('Time')
plt.ylabel('Population')
plt.legend(loc = "upper right");

In [ ]:
def n_update(n, mu):
    seeds = np.random.uniform(0, 1, np.int_(n))
    pop_update = np.zeros(5)
    for seed in seeds:
        if seed < mu:
            pop_update[0] += 1
    return pop_update

def s_update(s, mu, beta, beta_f, epsilon):
    seeds = np.random.uniform(0, 1, np.int_(s))
    pop_update = np.zeros(5)
    for seed in seeds:
        if seed < mu:
            pop_update[0] -= 1
        elif seed < (mu + beta + beta_f):
            pop_update[0] -= 1
            pop_update[2] += 1
        elif seed < (mu + beta + beta_f + epsilon):
            pop_update[0] -= 1
            pop_update[1] += 1
    return pop_update

def v_update(v, mu, phi, phi_f):
    seeds = np.random.uniform(0, 1, np.int_(v))
    pop_update = np.zeros(5)
    for seed in seeds:
        if seed < mu:
            pop_update[1] -= 1
        elif seed < (mu + phi + phi_f):
            pop_update[1] -= 1
            pop_update[2] += 1
    return pop_update

def i_update(i, mu, sigma, alpha, gamma):
    seeds = np.random.uniform(0, 1, np.int_(i))
    pop_update = np.zeros(5)
    for seed in seeds:
        if seed < mu:
            pop_update[2] -= 1
        elif seed < (mu + sigma + alpha):
            pop_update[2] -= 1
            pop_update[4] += 1
        elif seed < (mu + sigma + alpha + gamma):
            pop_update[2] -= 1
            pop_update[3] += 1
    return pop_update

def r_update(r, mu):
    seeds = np.random.uniform(0, 1, np.int_(r))
    pop_update = np.zeros(5)
    for seed in seeds:
        if seed < mu:
            pop_update[4] -= 1
    return pop_update

def f_update(f, mu):
    seeds = np.random.uniform(0, 1, np.int_(f))
    pop_update = np.zeros(5)
    for seed in seeds:
        if seed < mu:
            pop_update[3] -= 1
    return pop_update

In [ ]:
def svifr_model(init_pop, time, beta, beta_f, phi, phi_f, mu, gamma, sigma, alpha, epsilon):
    
    pop = np.zeros([5, time+1])
    pop[0,0] = init_pop
    for t in range(1, time+1):
        n = np.sum(pop, axis=0)[t-1]
        s = pop[0, t-1]
        v = pop[1, t-1]
        i = pop[2, t-1]
        f = pop[3, t-1]
        r = pop[4, t-1]

        n_change = n_update(n, mu)
        s_change = s_update(s, mu, beta, beta_f, epsilon)
        v_change = v_update(v, mu, phi, phi_f)
        i_change = i_update(i, mu, sigma, alpha, gamma)
        r_change = r_update(r, mu)
        f_change = f_update(f, mu)
        
        pop_change = n_change + s_change + v_change + i_change + r_change + f_change
        pop[:,t] = np.add(pop[:,t-1], pop_change)
        
    df = pd.DataFrame({
        'time': np.arange(time+1),
        's': pop[0,:],
        'v': pop[1,:],
        'i': pop[2,:],
        'r': pop[3,:],
        'f': pop[4,:],

    })
    return df

In [ ]:
svifr = svifr_model(1000, 100, beta, beta_f, phi, phi_f, mu, gamma, sigma, alpha, epsilon)
print(svifr)

In [ ]:
plt.figure(figsize=(9, 6))
sns.lineplot(data=svifr, x="time", y="s", label="Susceptible")
sns.lineplot(data=svifr, x="time", y="v", label="Vaccinated")
sns.lineplot(data=svifr, x="time", y="i", label="Infectious")
sns.lineplot(data=svifr, x="time", y="r", label="Removed")
sns.lineplot(data=svifr, x="time", y="f", label="Funeral")
plt.title('Stochastic Simulation of SVIFR Model')
plt.xlabel('Time')
plt.ylabel('Population')
plt.legend(loc = "upper right");